In [1]:
'''
오늘은 딥러닝 공부합니다.   0
내일은 프로젝트 합니다.     0
날씨가 흐립니다.            1
'''

'\n오늘은 딥러닝 공부합니다.   0\n내일은 프로젝트 합니다.     0\n날씨가 흐립니다.            1\n'

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

In [8]:
X_train.shape

(25000,)

In [9]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [6]:
len(X_train[0])

218

In [10]:
y_train[0] # 긍정 : 1, 부정 : 0

1

In [13]:
X_test.shape

(25000,)

In [14]:
# ex) 그 영화 너무 감동적이었어요. 주인공 정말 멋져요. 왕 추천!!!
# 1) 텍스트 전처리(구문분석, 형태소 분리, 불필요한 형태소 제거, 동음이의어, 다의어 처리)

# 은행 계좌를 만들려고 합니다
#             생성하려고
#             틀려고
#      구좌를 만들려고
# ...
# 영화 너무 웃겨서 배 아팠어요  => 배 50
# 영화 보면서 배를 먹었어요  => 배 50x , 15
# 전에 봤던 것보다 2배 이상 더 잼있었어요  => 배 50x, 2100
# ...
# -> 모델 성능이 떨어짐(배라는 단어 제대로 인식x)
# -> 다 구분하게 만들어야함(문맥파악)

# 영화 너무 웃겨서 배 아팠어요 => 영화 너무 웃기다 배 아프다 => 52 123 323 555 678

In [15]:
X_train = sequence.pad_sequences(X_train, maxlen=500)
X_test = sequence.pad_sequences(X_test, maxlen=500)

In [16]:
model = Sequential()
model.add(Embedding(5000, 100)) # 5000차원 -> 100차원
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1)) # filter 64, kernel_size 5, same : 크기 같게, valid : 크기가 줄어듬
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55)) # 55차원 결과 출력
model.add(Dense(1))
model.add(Activation('sigmoid')) # 합쳐서 써도 되고 이처럼 별도로 써도됨
model.summary()

# cnn, lstm 사용

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         500000    
                                                                 
 dropout (Dropout)           (None, None, 100)         0         
                                                                 
 conv1d (Conv1D)             (None, None, 64)          32064     
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 64)          0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 55)                26400     
                                                                 
 dense (Dense)               (None, 1)                 56        
                                                        

In [17]:
# 모델의 실행 옵션을 정합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 학습의 조기 중단을 설정합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)
# 모델을 실행합니다.
history = model.fit(X_train, y_train, batch_size=40, epochs=100, validation_split=0.25, callbacks=[early_stopping_callback])
# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))

Epoch 1/100
469/469 [==============================] - 101s 209ms/step - loss: 0.4070 - accuracy: 0.7962 - val_loss: 0.2821 - val_accuracy: 0.8824
Epoch 2/100
469/469 [==============================] - 96s 205ms/step - loss: 0.2365 - accuracy: 0.9083 - val_loss: 0.2636 - val_accuracy: 0.8942
Epoch 3/100
469/469 [==============================] - 93s 197ms/step - loss: 0.1900 - accuracy: 0.9285 - val_loss: 0.2790 - val_accuracy: 0.8866
Epoch 4/100
469/469 [==============================] - 92s 197ms/step - loss: 0.1530 - accuracy: 0.9447 - val_loss: 0.2841 - val_accuracy: 0.8840
Epoch 5/100
782/782 [==============================] - 30s 38ms/step - loss: 0.3357 - accuracy: 0.8769

 Test Accuracy: 0.8769
